# Responsável por tratar os dados pertinente a cliente

## Importando bibliotecas

In [1]:
import sys, os

# Carregando pacote que estao fora de /transform
path_raiz = os.path.abspath(os.path.join("..", ".."))
sys.path.append(path_raiz)

In [2]:
import pandas as pd
from src.utils import carregar_tabela_excel
from src.insert.insert_contract import inserir_contrato
import json
import datetime

Ambiente: test


## Importando e tratando os dados

In [3]:
# Obter a data e hora atual
now = datetime.datetime.now()

# Formatar a data e hora como 'YYYYMMDD_HHMMSS'
timestamp = now.strftime("%Y%m%d_%H%M%S")
timestamp

'20241226_130917'

In [4]:
source_input = "../../data/baseTesteIXC_Transform.xlsx"
source_output = f"../../src/inserted/cliente_insert_{timestamp}.xlsx"

In [5]:
# Carregar o DataFrame com as colunas especificadas como strings
df = pd.read_excel(source_input, dtype=str).fillna("")

In [6]:
# Lista das colunas que fazem sentido enviar para a API
colunas_para_api = [
    'razaoSocial',
    'Email',
    'Celular',
    'cpf_cnpj',
    'rg_ie',
    'filial_id',
    'idCidade',
    'Endereco',
    'Complemento',
    'Numero',
    'Bairro',
    'Cep',
]

# Filtra o DataFrame para manter apenas essas colunas
df = df[colunas_para_api]


### Inserção teste

In [ ]:
inserir_contrato(
    #tipo="I",
    id_cliente="117641",
    id_vd_contrato="6824",  # Plano do contrato
    id_tipo_contrato="5",  # Id tipo de cobrança
    id_modelo="179",  # Id modelo do contrato
    id_filial="35",
    id_vendedor="1",
    data_base_financeira="2024-04-01",
    contrato="IMPORTAÇÃO",
    id_carteira_cobranca="67"
    cep="57160-000"
)

200 OK
b'{"type":"error","message":"Preencha Cep","atualiza_campos":[{"tipo":"i","campo":"data_desistencia","valor":""},{"tipo":"s","campo":"status","valor":""},{"tipo":"i","campo":"id_cliente","valor":"117641"},{"tipo":"i","campo":"data_ativacao","valor":""},{"tipo":"d","campo":"data_renovacao","valor":""},{"tipo":"d","campo":"nao_avisar_ate","valor":""},{"tipo":"d","campo":"nao_bloquear_ate","valor":""},{"tipo":"d","campo":"data_cancelamento","valor":""},{"tipo":"d","campo":"dt_ult_bloq_manual","valor":""},{"tipo":"d","campo":"data_cadastro_sistema","valor":"2024-12-26"}]}'


<Response [200]>

## Inserção do DF

In [8]:
# Limitando o DF
df = df.head(10)

In [9]:
# Log para verificação de campo vazio
print(df.isnull().sum())

razaoSocial    0
Email          0
Celular        0
cpf_cnpj       0
rg_ie          0
filial_id      0
idCidade       0
Endereco       0
Complemento    0
Numero         0
Bairro         0
Cep            0
dtype: int64


In [10]:
# Crie uma cópia do DataFrame original para evitar o SettingWithCopyWarning
df = df.copy()

# Crie as colunas adicionais (inicialmente vazias)
df.loc[:, "idClienteIXC"] = None
df.loc[:, "statusInsercao"] = None
df.loc[:, "logRetorno"] = None

for idx, row in df.iterrows():
    # Mapeamos as colunas do DF para os parâmetros da função inserir_cliente.
    razao = row["razaoSocial"]
    email = row["Email"]
    celular = row["Celular"]
    cnpj_cpf = row["cpf_cnpj"]
    rg_ie = row["rg_ie"]
    filial = row["filial_id"]
    cidade = row["idCidade"] 
    endereco = row["Endereco"]
    complemento = row["Complemento"] if pd.notnull(row["Complemento"]) else ""
    numero = row["Numero"]
    bairro = row["Bairro"]
    cep = row["Cep"]
    
    # Chamamos a função inserir_cliente:
    response = inserir_cliente(
        razao=razao,
        cnpj_cpf=cnpj_cpf,
        cep=cep,
        endereco=endereco,
        numero=numero,
        complemento=complemento,
        bairro=bairro,
        cidade=cidade,
        filial_id=filial,
        telefone_celular=celular,
        email=email,
        ie_identidade=rg_ie,
        # Se você quiser passar outros campos com valores-padrão, inclua aqui
        # Ex.: uf="AL" etc.
    )
    
    status_code = response.status_code
    content = response.content  # bytes

    # Fazemos parse do JSON:
    # b'{"type":"success","message":"Registro inserido com sucesso!","id":"117580", ...}'
    json_resp = json.loads(content.decode("utf-8", errors="replace"))

    if status_code == 200 and json_resp.get("type") == "success":
        # Sucesso: capturamos o "id"
        client_id = json_resp.get("id")  # Exemplo: "117580"
        
        df.at[idx, "idClienteIXC"] = client_id
        df.at[idx, "statusInsercao"] = "sucesso"
        # logRetorno pode ficar vazio ou None em caso de sucesso
        df.at[idx, "logRetorno"] = json_resp
    else:
        # Falha: salvamos o log completo
        df.at[idx, "idClienteIXC"] = None
        df.at[idx, "statusInsercao"] = "falha"
        # Guardamos o JSON completo (ou a chave "message", se preferir):
        df.at[idx, "logRetorno"] = json_resp


KeyboardInterrupt



### Retorno

In [110]:
df.to_excel(source_output)

In [111]:
df

,razaoSocial,Email,Celular,cpf_cnpj,rg_ie,filial_id,idCidade,Endereco,Complemento,Numero,Bairro,Cep,idClienteIXC,statusInsercao,logRetorno
0,Abelanio Veríssimo Peixoto,teste@gmail.com,99999999999,685.796.740-77,Isento,35,1659,Vila São Sebastião,Próximo ao cacimbão,0,Centro,57830-000,117636,sucesso,"{'type': 'success', 'message': 'Registro inser..."
1,Aclecia da Silva,teste@gmail.com,99999999999,281.742.620-75,30234875,35,1659,Maria de Lurdes,Próximo ao mirante,SN,Centro,57830-000,117637,sucesso,"{'type': 'success', 'message': 'Registro inser..."
2,Adeilton da Mata,teste@gmail.com,99999999999,673.248.950-00,40259595,35,1659,Ass. Nova Esperança,próximo a casa de farinha,SN,Zona Rural,57830-000,117638,sucesso,"{'type': 'success', 'message': 'Registro inser..."
3,Adeilton da Silva,teste@gmail.com,99999999999,108.151.210-59,3614522-0,35,1659,Gravatá,Lote do Sr. Del,SN,Zona Rural,57830-000,117639,sucesso,"{'type': 'success', 'message': 'Registro inser..."
4,Adeilton Teixeira de Lima,teste@gmail.com,99999999999,976.069.770-03,36145220,35,1659,João Lyra / Qd: A,,24,Conj. Raimundo Nonato Lopes,57830-000,117640,sucesso,"{'type': 'success', 'message': 'Registro inser..."
5,Adelmo Luiz Berto De Oliveira,teste@gmail.com,99999999999,660.107.170-60,ISENTO,35,1659,Rua: Manoel Luiz da Silva,Próximo ao Mirante,SN,Centro,57830-000,117641,sucesso,"{'type': 'success', 'message': 'Registro inser..."
6,Ademar Manoel da Silva,teste@gmail.com,99999999999,52.459.301/0001-81,30972841,35,1659,Ass.: Flor do Mundaú,,SN,Zona Rural,57830-000,117642,sucesso,"{'type': 'success', 'message': 'Registro inser..."
7,Adriana Paulino da Silva,teste@gmail.com,99999999999,54.404.537/0001-46,ISENTO,35,1659,Platô 1/ Q: 02,,06,Conj. Raimundo Nonato Lopes,57830-000,117643,sucesso,"{'type': 'success', 'message': 'Registro inser..."
